[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/AnnemarieKeller/MLProject_CapstoneBlackBoxOptimisation/blob/architecture/dynamicloop.ipynb
)



In [13]:
import numpy as np
from scripts.BBOloop import *
from scripts import *
functionNo = 6
weekNo = 7
X, y = generate_data(1, 7)
config = funcConfig.FUNCTION_CONFIG[functionNo]

best_input, best_output, history, best_results = adaptive_bbo_dynamic_full(
    X_init=X,
    y_init=y,
    config=config,
    acquisition_list=["EI", "UCB", "PI", "THOMPSON"],
    kernel_list=["RBF", "Matern"],
    num_iterations=50,   #
    random_state=42,
    base_candidates=100,  # smaller for speed in example
    candidate_scale=50,
    scale_candidates=False
)

# --- Print results ---
print("\nBest input found:", best_input)
print("Best output:", best_output)

print("\nHistory of all evaluated points:")
print(history)

print("\nBest results per iteration:")
for r in best_results:
    print(r)


Initial count: 10
Weekly count: 7
Combined: 17
17


/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.14592369942847. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.14592369942847. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/si

/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.14592369942847. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.14592369942847. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/si

Iter 1 | Acq: UCB | Kernel: Matern | Next input: [0.14489487 0.48945276] | Predicted y: 0.000000 | Candidates: 200


/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.0877603470733765. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.0877603470733765. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.0877603470733765. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local

...

/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 5.838612845720547. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages

Iter 49 | Acq: UCB | Kernel: Matern | Next input: [0.00986384 0.97650931] | Predicted y: 0.000268 | Candidates: 296


/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 5.83428188316541. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 5.83428188316541. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/si

Iter 50 | Acq: UCB | Kernel: Matern | Next input: [0.03938212 0.97247111] | Predicted y: 0.000268 | Candidates: 298

Best input found: [0.02761638 0.96297263]
Best output: 0.00026817172271020466

History of all evaluated points:
{'X': array([[3.19403889e-01, 7.62959374e-01],
       [5.74329215e-01, 8.79898105e-01],
       [7.31023631e-01, 7.32999876e-01],
       [8.40353417e-01, 2.64731614e-01],
       [6.50114060e-01, 6.81526352e-01],
       [4.10437137e-01, 1.47554299e-01],
       [3.12691157e-01, 7.87227779e-02],
       [6.83418169e-01, 8.61057464e-01],
       [8.25072518e-02, 4.03487506e-01],
       [8.83889829e-01, 5.82253974e-01],
       [2.02514000e-01, 5.86723000e-01],
       [4.89595000e-01, 1.16877000e-01],
       [2.00260000e-02, 9.96624000e-01],
       [9.56717000e-01, 4.35900000e-03],
       [9.85825000e-01, 9.86810000e-01],
       [4.47720000e-02, 9.31757000e-01],
       [9.51495000e-01, 7.45322000e-01],
       [1.44894872e-01, 4.89452760e-01],
       [8.17443562e-01, 3.4

/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 5.83428188316541. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [5]:
import os
import sys


current_dir = os.getcwd()
repo_root = current_dir

while "scripts" not in os.listdir(repo_root):
    parent = os.path.abspath(os.path.join(repo_root, ".."))
    if parent == repo_root:  # reached filesystem root
        raise FileNotFoundError("Cannot find 'scripts' folder in parent directories")
    repo_root = parent

# Add repo root to Python path
if repo_root not in sys.path:
    sys.path.append(repo_root)

print("!Added repo root to sys.path:", repo_root)

import scripts
import scripts.functionConfig as funcConfig
import scripts.accquistions as acq
import scripts.setBoundary as bound
import scripts.BBOloop as bbo_loop
from scripts.BBOloop import *
from scripts import *
import scripts.functionConfig as funcConfig
import scripts.accquistions as acq
import scripts.setup.gpBuilder as build
from scripts.utils.generateX_Y import *
from scripts.BBOloop import *
import scripts.functionConfig as funcConfig
from collections import defaultdict
from scripts.utils.utils import *

!Added repo root to sys.path: /workspaces/MLProject_CapstoneBlackBoxOptimisation


In [ ]:
# -----------------------------
# Automated BBO Runner with History
# -----------------------------
def run_all_bbo_with_history(functions, weekNo, bbo_methods, config, num_iterations=30):
    """
    Run multiple BBO methods on multiple functions automatically,
    storing full history as well as best results.

    Args:
        functions: dict of {name: callable} 
        bbo_methods: list of method names (strings)
        config: config dict for BBO loops
        num_iterations: number of iterations per BBO loop

    Returns:
        results: dict storing best_x, best_y, and full history per function per method
    """
    results = {}

    for fname, f in functions.items():
        print(f"\n=== Running function: {fname} ===")
        results[fname] = {}
        # Generate initial training data
        X_init, y_init = generate_data(f, weekNo)  

        for method in bbo_methods:
            print(f"\nRunning method: {method}")
            if method == "GP_BBO":
                best_x, best_y, history = function_free_bbo_multi_acq(
                    X_init, y_init, config, num_iterations=num_iterations
                )
            elif method == "SVR_BBO":
                best_x, best_y, history = svr_bbo_loop(
                    X_init, y_init, config, n_iterations=num_iterations
                )
            elif method == "Dynamic_BBO":
                best_x, best_y, history = adaptive_bbo_dynamic(
                    X_init, y_init, config, num_iterations=num_iterations
                )
            else:
                raise ValueError(f"Unknown method: {method}")

            # Store results including full history
            results[fname][method] = {
                "best_x": best_x,
                "best_y": best_y,
                "history": history
            }

            print(f" -> Done {method}: best_y = {best_y:.4f}, best_x = {best_x}")

    # Summary: pick best method per function
    print("\n=== Summary of Best Results ===")
    for fname, res in results.items():
        best_method = max(res, key=lambda m: res[m]["best_y"])
        best_y = res[best_method]["best_y"]
        best_x = res[best_method]["best_x"]
        print(f"Function {fname} -> Best method: {best_method}, best_y: {best_y:.4f}, best_x: {best_x}")

    return results


In [2]:
from scripts.utils.generateX_Y import *
from scripts.BBOloop import *
import scripts.functionConfig as funcConfig
from collections import defaultdict

weekno = 7
results = defaultdict(dict)

for iter_num, cfg in funcConfig.FUNCTION_CONFIG.items():
    print(f"Function {iter_num} -> {cfg['name']}, dim={cfg['dim']}, acquisition={cfg['acquisition']}")
    
    # Generate initial data
    X_train, y_train = generate_data(iter_num, weekno)
    
    # Run adaptive BBO
    best_input, best_output, history, best_results = adaptive_bbo_dynamic_full(
        X_train, y_train, cfg, scale_candidates=False
    )
    
    print(f"Function {iter_num} finished. Best output: {best_output} Best Input {best_input}")
    # Ensure the outer dict exists
    results.setdefault(iter_num, {})

# Find the iteration with the overall best output
    best_idx = np.argmax([r["best_output"] for r in best_results])
    best_acq_name = best_results[best_idx]["acquisition"]

# Store using acquisition name as the key
    results[iter_num][best_acq_name] = {
        "best_x": best_input,
        "best_y": best_output,
        "history": history,
        "best_results": best_results
    }

    
save_weekly_results(results,weekno)
for iter_num, res in results.items():
    fname = funcConfig.FUNCTION_CONFIG[iter_num]["name"]
    best_method = max(res, key=lambda m: res[m]["best_y"])
    best_y = res[best_method]["best_y"]
    best_x = res[best_method]["best_x"]
    print(f"Function {fname} -> Best method: {best_method}, best_y: {best_y:.6f}, best_x: {best_x}")


Function 1 -> 2D Contamination, dim=2, acquisition=UCB
Initial count: 10
Weekly count: 7
Combined: 17
17


/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.14592369942847. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.14592369942847. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.14592369942847. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.14592369942847. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/si

Iter 1 | Acq: THOMPSON | Kernel: Matern | Next input: [0.83213418 0.50746773] | Predicted y: 0.000000 | Candidates: 900


/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.019339963158136. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 6.019339963158136. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/

...

/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 5 of parameter k1__length_scale is close to the specified upper bound 10.778123909068649. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-package

Iter 29 | Acq: UCB | Kernel: Matern | Next input: [0.17076173 0.2621202  0.15155048 0.05016885 0.83910832 0.64426205
 0.25101278 0.63235392] | Predicted y: 10.089581 | Candidates: 3593


/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 5 of parameter k1__length_scale is close to the specified upper bound 10.725026118681642. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 7 of parameter k1__length_scale is close to the specified upper bound 10.725026118681642. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.1

Iter 30 | Acq: UCB | Kernel: Matern | Next input: [0.33121753 0.01119836 0.27668251 0.01815842 0.7126931  0.45338457
 0.1150034  0.70659391] | Predicted y: 10.089581 | Candidates: 3646
Function 8 finished. Best output: 10.089580545921546 Best Input [0.12152155 0.15069884 0.20905791 0.10310349 0.98556055 0.84813086
 0.2473937  0.20359781]
CSV file 'overall_best_results.csv' created.
Function 2D Contamination -> Best method: UCB, best_y: 0.000455, best_x: [0.6980914  0.65626393]
Function 2D Noisy Log-Likelihood -> Best method: UCB, best_y: 0.684085, best_x: [0.702637 0.926564]
Function 3D Drug Combination -> Best method: THOMPSON, best_y: -0.010965, best_x: [0.5963609  0.51114014 0.85691668]
Function 4D Warehouse Placement -> Best method: UCB, best_y: 0.581717, best_x: [0.406239 0.409937 0.37737  0.422212]
Function 4D Chemical Yield -> Best method: UCB, best_y: 3834.610065, best_x: [0.00926011 0.69903927 0.78677716 0.99868203]
Function 5D Cake Recipe -> Best method: PI, best_y: -0.144393

/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 5 of parameter k1__length_scale is close to the specified upper bound 10.725026118681642. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 7 of parameter k1__length_scale is close to the specified upper bound 10.725026118681642. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [3]:

    


for iter_num, res in results.items():
    fname = funcConfig.FUNCTION_CONFIG[iter_num]["name"]
    best_method = max(res, key=lambda m: res[m]["best_y"])
    best_y = res[best_method]["best_y"]
    best_x = res[best_method]["best_x"]
    best_x_str = ", ".join([f"{v:.6f}" for v in best_x])
    print(f"Function {fname} -> Best method: {best_method}, best_y: {best_y:.6f}, best_x: [{best_x_str}]")



Function 2D Contamination -> Best method: UCB, best_y: 0.000455, best_x: [0.698091, 0.656264]
Function 2D Noisy Log-Likelihood -> Best method: UCB, best_y: 0.684085, best_x: [0.702637, 0.926564]
Function 3D Drug Combination -> Best method: THOMPSON, best_y: -0.010965, best_x: [0.596361, 0.511140, 0.856917]
Function 4D Warehouse Placement -> Best method: UCB, best_y: 0.581717, best_x: [0.406239, 0.409937, 0.377370, 0.422212]
Function 4D Chemical Yield -> Best method: UCB, best_y: 3834.610065, best_x: [0.009260, 0.699039, 0.786777, 0.998682]
Function 5D Cake Recipe -> Best method: PI, best_y: -0.144393, best_x: [0.488319, 0.333366, 0.536917, 0.820112, 0.058801]
Function 6D ML Hyperparameters -> Best method: UCB, best_y: 1.902213, best_x: [0.025094, 0.105987, 0.131162, 0.177208, 0.382072, 0.610314]
Function  8D ML Hyperparameters -> Best method: UCB, best_y: 10.089581, best_x: [0.121522, 0.150699, 0.209058, 0.103103, 0.985561, 0.848131, 0.247394, 0.203598]


In [ ]:
for func_name, iter_results in results.items():
    # Determine if iter_results is a dict or list
    if isinstance(iter_results, dict):
        iter_results_list = iter_results.values()  # dict -> take values
    else:
        iter_results_list = iter_results      # already a list

    # Initialize best trackers
    best_y = -float("inf")
    best_x = None
    best_kernel = None
    best_kernel_params = {}
    best_acq = None

    # Loop over iterations to find overall best
    for res in iter_results_list:
        if res["best_output"] > best_y:
            best_y = res["best_output"]
            best_x = res["best_input"]
            best_kernel = res.get("kernel", "N/A")
            best_kernel_params = res.get("kernel_params", {})
            best_acq = res.get("acquisition", "N/A")

    # Print nicely
    print(f"Function: {func_name}")
    print(f"  Best output (y): {best_y:.6f}")
    print(f"  Best input (x): {[f'{xi:.6f}' for xi in best_x]}")
    print(f"  Kernel: {best_kernel}")
    print(f"  Kernel parameters: {best_kernel_params}")
    print(f"  Acquisition: {best_acq}")
    print("-" * 60)


In [ ]:

for iter_num, res in results.items():
    fname = funcConfig.FUNCTION_CONFIG[iter_num]["name"]
    best_method = max(res, key=lambda m: res[m]["best_y"])
    best_y = res[best_method]["best_y"]
    best_x = res[best_method]["best_x"]
    print(f"Function {fname} -> Best method: {best_method}, best_y: {best_y:.6f}, best_x: {best_x}")
    print( "-".join(f"{x:.6f}" for x in best_x))

In [ ]:
for iter_num, res in results.items():
  
    
    # pick best method using their best_y values
    best_method = max(res, key=lambda m: res[m]["best_y"])
    entry = res[best_method]

    best_y = entry["best_y"]
    best_x = entry["best_x"]
    kernel_name = entry.get("kernel", "N/A")
    kernel_params = entry.get("kernel_params", {})
    acq_method = entry.get("acquisition", "N/A")
    
    print(f"\nFunction: ")
    print(f"  Best method:        {best_method}")
    print(f"  Acquisition:        {acq_method}")
    print(f"  Kernel:             {kernel_name}")
    print(f"  Kernel parameters:  {kernel_params}")
    print(f"  Best y:             {best_y:.6f}")
    print(f"  Best x:             {best_x}")
    print("  Best x formatted:   " + "-".join(f"{x:.6f}" for x in best_x))
    print(res)


In [ ]:
for func_num, iter_results in results.items():
    best_y = -float("inf")
    best_x = None
    best_method = None
    best_kernel = None
    best_kernel_params = None
    best_acq = None

    for res in iter_results:  # iter_results is a list of dicts
        if res["best_output"] > best_y:
            best_y = res["best_output"]
            best_x = res["best_input"]
            best_method = res.get("acquisition", "N/A")
            best_kernel = res.get("kernel", "N/A")
            best_kernel_params = res.get("kernel_params", {})
            best_acq = res.get("acquisition", "N/A")

    fname = funcConfig.FUNCTION_CONFIG[func_num]["name"]
    print(f"Function: {fname}")
    print(f"  Best method: {best_method}")
    print(f"  Acquisition: {best_acq}")
    print(f"  Kernel: {best_kernel}")
    print(f"  Kernel parameters: {best_kernel_params}")
    print(f"  Best y: {best_y:.6f}")
    print("  Best x: " + " - ".join(f"{x:.6f}" for x in best_x))
    print("-" * 50)




In [ ]:

    # Generate initial data
X_train, y_train = generate_data(5, 6)
print(X_train.shape[1], "shape")
cfg = funcConfig.FUNCTION_CONFIG[7]
    
    # Run adaptive BBO
best_input, best_output, history, best_results = adaptive_bbo_dynamic_full(
        X_train, y_train, cfg, scale_candidates=False
)
print(f"Function {cfg.name} finished. Best output: {best_output} Best Input {best_input}")

In [ ]:

import scripts
import scripts.functionConfig as funcConfig
import scripts.accquistions as acq
import scripts.setBoundary as bound
import scripts.setup.gpBuilder as build


import scripts.BBOloop as bbo_loop  